In [1]:
%reload_ext autoreload
%autoreload 2
from src.acnets.pipeline import Parcellation
from src.acnets.pipeline import TimeseriesAggregator, ConnectivityExtractor
from src.acnets.pipeline import ConnectivityAggregator
import pandas as pd
import numpy as np

In [27]:
bids_dir = 'data/julia2018'
parcellation_cache_dir = 'data/julia2018/derivatives/resting_timeseries/'

# parcellation
parcellation = Parcellation(
    atlas_name='dosenbach2010',
    bids_dir=bids_dir,
    cache_dir=parcellation_cache_dir)
dataset = parcellation.fit_transform(None)

# time-series aggregation
ts_agg = TimeseriesAggregator(strategy='region')
dataset = ts_agg.fit_transform(dataset)

# connectivity extraction
conn_extractor = ConnectivityExtractor(kind='correlation')
dataset = conn_extractor.fit_transform(dataset)

# connectivity aggregation
conn_agg = ConnectivityAggregator()
dataset = conn_agg.fit_transform(dataset)
dataset['network'].to_pandas()
conn = dataset['connectivity'].to_dataframe().reset_index()
conn

,subject,region_src,region_dst,connectivity
0,AVGP01,vmPFC 1,vmPFC 1,1.000000
1,AVGP01,vmPFC 1,aPFC 2,0.384150
2,AVGP01,vmPFC 1,aPFC 3,0.081446
3,AVGP01,vmPFC 1,mPFC 4,0.443102
4,AVGP01,vmPFC 1,aPFC 5,0.454185
...,...,...,...,...
819195,NVGP19NEW,post occipital 160,post occipital 156,0.127798
819196,NVGP19NEW,post occipital 160,post occipital 157,0.411767
819197,NVGP19NEW,post occipital 160,post occipital 158,0.482605
819198,NVGP19NEW,post occipital 160,post occipital 159,0.029156
